### Pix2Pix modeling

Model repo: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Paper: https://arxiv.org/pdf/1611.07004.pdf

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from cellvision_lib import train_test_val
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Proprocesing the data for pix2pix model

MAX_DEPTH = 100
NUM_SAMPLES = 109

folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized'

train, test, val = train_test_val(folder_path, channels = 1, train_pp = .67, test_pp = .165, val_pp = .165, set_seed = 1)
train[0:10]




# print(len(channel1_comps))

[['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample68_channel1_z93.tif',
  '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample68_channel6_z93.tif'],
 ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample57_channel1_z33.tif',
  '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample57_channel6_z33.tif'],
 ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample74_channel1_z35.tif',
  '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample74_channel6_z35.tif'],
 ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample60_channel6_z97.tif',
  '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample60_channel1_z97.tif'],
 ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample59_channel1_z9.tif',
  '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsample59_channel6_z9.tif'],
 ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalizedsa